# Database Pipeline

Load optimized chunks with embeddings from JSON and store them in PostgreSQL with pgvector support for semantic search.

**Features:**
- Load chunks from chunks_embedded.json with parent-child relationships
- Optimized database schema (only essential fields)
- Store demo code as JSONB for easy retrieval
- Create pgvector extension for vector similarity search
- Parent-child indexing for hierarchical document retrieval
- Automatic statistics tracking

## Database Setup

Before using this module, ensure PostgreSQL is configured:

```sql
-- Create pgvector extension
CREATE EXTENSION IF NOT EXISTS vector;

-- Create database
CREATE DATABASE marigold_rag;
```

## Section 1: Database Class

Defines the `DB` dataclass for PostgreSQL connection management and chunk insertion with pgvector support.

In [5]:
import json, psycopg2
from dataclasses import dataclass

@dataclass
class DB:
    host: str = "localhost"
    port: int = 5432
    database: str = "marigold_rag"
    user: str = "postgres"
    password: str = "postgres"
    table_name: str = "chunks"  # Can be "chunks" or "chunks_primitive"
    
    def __post_init__(self):
        self.connection = psycopg2.connect(
            host=self.host, 
            port=self.port, 
            database=self.database, 
            user=self.user, 
            password=self.password,
            connect_timeout=5
        )
        self.cursor = self.connection.cursor()
    
    def setup(self):
        """Create pgvector extension and chunks table"""
        self.cursor.execute("CREATE EXTENSION IF NOT EXISTS vector")
        
        # Drop table if exists (to recreate with new data)
        self.cursor.execute(f"DROP TABLE IF EXISTS {self.table_name} CASCADE")
        
        self.cursor.execute(f"""
            CREATE TABLE IF NOT EXISTS {self.table_name} (
                id SERIAL PRIMARY KEY,
                component VARCHAR(255) NOT NULL,
                section_path TEXT NOT NULL,
                content TEXT NOT NULL,
                demo_code JSONB,
                parent_id INT,
                level INT DEFAULT 0,
                embedding vector(768) NOT NULL,
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
            )
        """)
        
        # Create indexes for efficient search
        self.cursor.execute(f"""
            CREATE INDEX IF NOT EXISTS {self.table_name}_embedding_idx 
            ON {self.table_name} USING ivfflat (embedding vector_cosine_ops) 
            WITH (lists=100)
        """)
        
        # Index for parent-child relationships
        self.cursor.execute(f"""
            CREATE INDEX IF NOT EXISTS {self.table_name}_parent_id_idx 
            ON {self.table_name}(parent_id)
        """)
        
        # Index for component search
        self.cursor.execute(f"""
            CREATE INDEX IF NOT EXISTS {self.table_name}_component_idx 
            ON {self.table_name}(component)
        """)
        
        self.connection.commit()
        print(f"Database table '{self.table_name}' created successfully")
    
    def insert(self, **chunk):
        """Insert chunk with embedding into database"""
        embedding_str = "[" + ",".join(str(x) for x in chunk['embedding']) + "]"
        
        self.cursor.execute(f"""
            INSERT INTO {self.table_name}
            (component, section_path, content, demo_code, parent_id, level, embedding) 
            VALUES (%s, %s, %s, %s, %s, %s, %s)
        """, (
            chunk.get('component'),
            chunk.get('section_path'),
            chunk.get('content'),
            json.dumps(chunk.get('demo_code', {})),
            chunk.get('parent_id'),
            chunk.get('level', 0),
            embedding_str
        ))
        self.connection.commit()
    
    def close(self):
        self.cursor.close()
        self.connection.close()

## Section 2: Load & Insert Chunks

Load chunks with embeddings from JSON files and insert them into PostgreSQL. Both semantic and primitive chunks are loaded, then you can execute one of the insert options below.

In [ ]:
from pathlib import Path
import json

CHUNKS_DIR = Path.cwd().parent.parent.parent / 'etl/data/chunks'

# Load semantic chunks
semantic_chunks_file = CHUNKS_DIR / 'chunks_embedded.json'
if semantic_chunks_file.exists():
    with open(semantic_chunks_file) as f:
        semantic_chunks = json.load(f)
else:
    semantic_chunks = None

# Load primitive chunks
primitive_chunks_file = CHUNKS_DIR / 'chunks_primitive_embedded.json'
if primitive_chunks_file.exists():
    with open(primitive_chunks_file) as f:
        primitive_chunks = json.load(f)
else:
    primitive_chunks = None

print(f"Semantic chunks loaded: {len(semantic_chunks) if semantic_chunks else 0}")
print(f"Primitive chunks loaded: {len(primitive_chunks) if primitive_chunks else 0}")


Semantic chunks loaded: 606
Primitive chunks loaded: 186


### Insert into Database - Choose One Option

- **OPTION A:** Insert semantic chunks with parent-child relationships
- **OPTION B:** Insert primitive chunks (flat structure)

In [ ]:
"""Option A: Insert semantic chunks into DB"""

if semantic_chunks is not None:
    db = DB(
        host="localhost",
        port=5432,
        database="marigold_rag",
        user="postgres",
        password="postgres",
        table_name="chunks"
    )
    
    print(f"\nConnecting to: {db.host}:{db.port}/{db.database}")
    print(f"Table: {db.table_name}\n")
    
    try:
        db.setup()
        print("\nInserting semantic chunks into database...")
        for i, chunk in enumerate(semantic_chunks, 1):
            db.insert(**chunk)
            if i % 100 == 0:
                print(f" Inserted {i} chunks...")
        
        print(f"\nSuccessfully stored {len(semantic_chunks)} semantic chunks in database")
        
        db.cursor.execute(f"SELECT COUNT(*) FROM {db.table_name}")
        
        db.cursor.execute(f"SELECT COUNT(DISTINCT component) FROM {db.table_name}")
        
        db.cursor.execute(f"SELECT COUNT(*) FROM {db.table_name} WHERE parent_id IS NOT NULL")
        
        db.cursor.execute(f"SELECT COUNT(*) FROM {db.table_name} WHERE demo_code IS NOT NULL AND demo_code != '{{}}'::jsonb")
        
    except Exception as e:
        print(f" Error: {e}")
        import traceback
        traceback.print_exc()
    finally:
        db.close()
        print("\nDatabase connection closed")
else:
    print("No semantic chunks loaded. Please run the load chunks cell first.")



Connecting to: localhost:5432/marigold_rag
Table: chunks



In [1]:
"""Option B: Insert primitive chunks into DB"""

if primitive_chunks is not None:
    db = DB(
        host="localhost",
        port=5432,
        database="marigold_rag",
        user="postgres",
        password="postgres",
        table_name="chunks_primitive"
    )
    
    print(f"\nConnecting to: {db.host}:{db.port}/{db.database}")
    print(f"Table: {db.table_name}\n")
    
    try:
        db.setup()

        for i, chunk in enumerate(primitive_chunks, 1):
            db.insert(**chunk)
        
        print(f"\nSuccessfully stored {len(primitive_chunks)} primitive chunks in database")
        
        db.cursor.execute(f"SELECT COUNT(*) FROM {db.table_name}")
        
        db.cursor.execute(f"SELECT COUNT(DISTINCT component) FROM {db.table_name}")
        
        db.cursor.execute(f"SELECT COUNT(*) FROM {db.table_name} WHERE demo_code IS NOT NULL AND demo_code != '{{}}'::jsonb")

        
    except Exception as e:
        print(f"Error: {e}")
        import traceback
        traceback.print_exc()
    finally:
        db.close()
        print(f"\nDatabase connection closed")
else:
    print("No primitive chunks loaded. Please run the load chunks cell first.")


NameError: name 'primitive_chunks' is not defined